# Joins and aggregations across Azure Cosmos DB collections using Azure Synapse Link 
In this notebook, we'll 

1. Create Spark tables pointing to [Azure Cosmos DB Analytical store](https://review.docs.microsoft.com/en-us/azure/cosmos-db/analytical-store-introduction?branch=release-build-cosmosdb) collections
2. Perform Joins, filters and aggregations across collections using Spark-SQL
3. Visualize the data using plotly and displayHTML()

>**Did you know?**  [Azure Synapse Link for Azure Cosmos DB](https://review.docs.microsoft.com/en-us/azure/cosmos-db/synapse-link?branch=release-build-cosmosdb) is a hybrid transactional and analytical processing (HTAP) capability that enables you to run near real-time analytics over operational data in Azure Cosmos DB.
&nbsp;

>**Did you know?**  [Azure Cosmos DB analytical store](https://review.docs.microsoft.com/en-us/azure/cosmos-db/analytical-store-introduction?branch=release-build-cosmosdb) is a fully isolated column store for enabling large scale analytics against operational data in your Azure Cosmos DB, without any impact to your transactional workloads.
&nbsp;

>**Did you know?**  Spark tables created on top of Azure Cosmos DB Collections are metadata tables, passing the queries to the corresponding collections.


### 1. Create Spark tables pointing to the Azure Cosmos DB Analytical Store collections using Azure Synapse Link 
>**Did you know?**  "cosmos.olap" is the Synapse Link Spark format that enables connection to the Azure Cosmos DB Analytical store collections.

>**Did you know?**  The Cosmos DB Linked Service in Synapse abstracts the connection configs for the Azure Cosmos DB Database. 

>[Click here](https://review.docs.microsoft.com/en-us/azure/synapse-analytics/synapse-link/how-to-connect-synapse-link-cosmos-db?branch=release-build-synapse#connect-an-azure-cosmos-db-database-to-a-synapse-workspace) to learn how to setup a Linked Service to connect Azure Cosmos DB database to the Azure Synapse workspace

In [ ]:
%%sql
create database CosmosDBIoTDemo

In [ ]:
%%sql

create table if not exists CosmosDBIoTDemo.IoTSignals
using cosmos.olap
options(spark.synapse.linkedService 'CosmosDBIoTDemo',
        spark.cosmos.container 'IoTSignals')


In [ ]:
%%sql

create table if not exists CosmosDBIoTDemo.IoTDeviceInfo
using cosmos.olap
options(spark.synapse.linkedService 'CosmosDBIoTDemo',
        spark.cosmos.container 'IoTDeviceInfo')


### 2. Perform Joins across collections, apply filters and aggregations using Spark SQL 

>**Did you know?**  Queries against Cosmos DB Analytical store is served independent of the transactional workload traffic without consuming any of the throughput provisioned for your operational data.

>**Did you know?**  Cosmos DB Analytical store uses columnar storage and is optimized for large-scale analytics workloads.

>**Did you know?**  Cosmos DB analytical store follows a cost effective consumption-based pricing model, which is based on data storage and analytical read/write operationsand queries executed and doesn’t require you to provision any throughput.

In [6]:
df_RPM_details = spark.sql("select a.deviceid \
                                 , b.devicetype \
                                 , cast(b.location as string) as location\
                                 , cast(b.latitude as float) as latitude\
                                 , cast(b.longitude as float) as  longitude\
                                 , a.measuretype \
                                 , a.unitSymbol \
                                 , cast(sum(measureValue) as float) as measureValueSum \
                                 , count(*) as count \
                            from CosmosDBIoTDemo.IoTSignals a \
                            left join CosmosDBIoTDemo.IoTDeviceInfo b \
                            on a.deviceid = b.deviceid \
                            where a.unitSymbol = 'RPM' \
                            group by a.deviceid, b.devicetype, b.location, b.latitude, b.longitude, a.measuretype, a.unitSymbol")

### View or chart the result-set using display() function


In [ ]:
display(df_RPM_details)

### 3. Visualizations using plotly and displayHTML()
The below shows a heatmap of IoT signals across diffrent locations

In [ ]:
from plotly.offline import plot
import plotly.express as px

df_RPM_details_pd = df_RPM_details.toPandas()
fig = px.scatter_mapbox(df_RPM_details_pd, 
                        lat='latitude', 
                        lon='longitude', 
                        size = 'measureValueSum',
                        color = 'measureValueSum',
                        hover_name = 'location',
                        hover_data = ['measureValueSum','location'],
                        size_max = 30,
                        color_continuous_scale = px.colors.carto.Temps,
                        zoom=3,
                        height=600,
                        width =900)

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

p = plot(fig,output_type='div')
displayHTML(p)       

<img src="https://revin.blob.core.windows.net/synapselinknotebooks/RPMHeatmap.PNG" width="1400" style="float: center;"/>
